<a href="https://colab.research.google.com/github/arteagac/absim/blob/main/simulation_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib nbagg
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
from matplotlib.patches import Circle
from matplotlib.patches import Arrow
from matplotlib.lines import Line2D
from scipy.spatial import distance_matrix
FPS = 20

In [ ]:
def distance(a, b):
    "a and b are arrays of shape (2, )"
    return np.sqrt(np.sum((np.power(a-b, 2))))

def signed_angle(a, b):
    """Angle in radians. a and b are arrays of shape (2, )"""
    return np.arctan2(a[0]*b[1] - a[1]*b[0], a.dot(b))

def normalize(a):
    """Normalizes vector to unit length. a has shape (2, 0)"""
    return a/np.sqrt(a.dot(a))

class PathPoint:
    def __init__(self, position, radius):
        self.position = position
        self.radius = radius
    
    def render_shapes(self):
        return [Circle(self.position, self.radius, color='r', fill=False)]

class Wall:
    def __init__(self, points):
        self.start, self.end = points[0], points[1]
    
    def render_shapes(self):
        return [Line2D((self.start[0], self.end[0]), (self.start[1], self.end[1]))]

class Civilian:
    def __init__(self, position):
        self.position = position
        self.radius = .5
        self.rotation = 0
        self.desired_speed = 1.29
        self.path = None
        self.next_path_point = 0

    def get_next_target(self):
        if self.next_path_point == len(self.path):  # Completed path
            return None
        curr_target = self.path[self.next_path_point]
        dist_to_curr_target = distance(self.position, curr_target.position)
        if dist_to_curr_target < curr_target.radius:  # Reached a new path point
            self.next_path_point += 1
        return self.path[self.next_path_point].position

    def move_basic(self):
        delta_time = 1.0/FPS
        next_target = self.get_next_target()
        if next_target is not None:
            self.rotation = signed_angle(self.position, next_target)
            velocity = self.desired_speed*normalize(self.position - next_target)
            self.position += velocity*delta_time
            

    def render_shapes(self):
        direc = self.position + np.array([self.radius*np.cos(self.rotation), self.radius*np.sin(self.rotation)])
        return [Circle(self.position, self.radius, color='g'),
                Line2D((self.position[0], direc[0]), (self.position[1], direc[1]), color='w')]

In [ ]:
# Walls 
hw = 4
walls_pos = np.array([
                      [[10, 10], [50, 10]],
                      [[50, 10], [50, 50]],
                      [[50, 50], [10, 50]],
                      [[10, 10 - hw], [50 + hw, 10 - hw]],
                      [[50 + hw, 10 - hw], [50 + hw, 50 + hw]],
                      [[50 + hw, 50 + hw], [10, 50 + hw]],                      
                      ]).astype(float)

walls = [Wall(wpos) for wpos in walls_pos]

# Path points
ppoints_pos = np.array([[10,  10 - hw/2],
                        [50 + hw/2, 10 - hw/2],
                        [50 + hw/2, 50 + hw/2],
                        [10, 50 + hw/2],
                        [0, 60]
                        ]).astype(float)

ppoints = [PathPoint(ppos, .7*(hw/2)) for ppos in ppoints_pos]
n_civilians = 30
civilians_pos = np.array([5, 5]) + np.random.randint(-5, 5, (n_civilians, 2)).astype(float)
civilians = [Civilian(cpos) for cpos in civilians_pos]
for civilian in civilians:
    civilian.path = ppoints

static_elements = walls + ppoints
dynamic_elements = civilians

def tick():
    for civilian in civilians:
        civilian.move_basic()

fig = plt.figure(figsize=(10, 10))
rax = fig.add_subplot(1, 1, 1, aspect=1) 
rax.set_xlim([0, 60])
rax.set_ylim([0, 60])
rax.grid()

#for elem in static_elements + dynamic_elements:
#    for shape in elem.render_shapes():
#        rax.add_artist(shape)

In [ ]:
#------- ANIMATE --------
sim_time = 5 # seconds

fig = plt.figure(figsize=(10, 10))
rax = fig.add_subplot(1, 1, 1, aspect=1) 
rax.set_xlim([0, 60])
rax.set_ylim([0, 60])
rax.grid()


for elem in static_elements:
    for shape in elem.render_shapes():
        rax.add_artist(shape)

#def animate_init():
#    return [shape for shape in elem.render_shapes() for elem in dynamic_elements]
dynamic_artists = [shape for shape in elem.render_shapes() for elem in dynamic_elements]

def copy_shape_data(source, dest):
    if isinstance(source, Line2D):
        dest_shape.set_data(source.xdata)

def animate_update(frame):
    tick()
    return [shape for shape in elem.render_shapes() for elem in dynamic_elements]

anim = animation.FuncAnimation(fig, animate_update, frames=sim_time*FPS, #init_func=animate_init,
                               interval=1000//FPS, blit=False)

HTML(anim.to_html5_video())

In [ ]:
%matplotlib nbagg
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML, Image 
from matplotlib.patches import Circle

fig = plt.figure(figsize=(10, 10))
rax = fig.add_subplot(1, 1, 1, aspect=1) 
rax.set_xlim([0, 800])
rax.set_ylim([0, 800])
y = 20

circles = [Circle([j*1, y], 10) for j in range(600)]
artists = [rax.add_artist(circle) for circle in circles]

def init():
    return artists

def update(i):
    for circle in circles:
        circle.center = (circle.center[0] , i+20)
    #y += 1
    return artists

anim = animation.FuncAnimation(fig, update, 
                               init_func=init, 
                               frames=100, 
                               interval=50,
                               blit=False)

HTML(anim.to_html5_video())


In [19]:
from IPython.display import Javascript
from time import sleep
def target_func(comm, msg):
    print(msg['content']['data'])
    comm.send({'foo': 2})

get_ipython().kernel.comm_manager.register_target('comm_target', target_func)

Javascript('''
(async () => {
  const channel = await google.colab.kernel.comms.open('comm_target', 'the data', []);
  console.log(Object.keys(channel));
    for await (const message of channel.messages) {
        console.log(message);
        channel.send({'ack': 1})
    }
})()
''')

<IPython.core.display.Javascript object>

the data


In [49]:
from IPython.display import display, HTML, Javascript
display(Javascript("google.colab.kernel.comms.registerTarget('absim', (comm, msg) => {console.log(msg.data)});"))

from ipykernel.comm import Comm
def send(comm, data):
    comm._publish_msg('comm_open', data=data, target_name=comm.target_name, target_module=comm.target_module)
    comm._closed = False
# Use comm to send a message from the kernel
comm = Comm(target_name='absim')
send(comm, {'foo': 'hi from python'})
comm.close()

<IPython.core.display.Javascript object>

In [ ]:
def draw_circle(center, radius, fill_color='green', line_color='#003300', line_width=1):
    html = f"""
    g.beginPath();
    g.arc({center[0]}, {center[1]}, {radius}, 0, 2 * Math.PI, false);
    g.fillStyle = '{fill_color}';
    g.fill();
    g.lineWidth = {line_width};
    g.strokeStyle = '{line_color}';
    g.stroke();
    """
    return html

from IPython.display import display, HTML, Javascript
from time import sleep

cwidth, cheight = 800, 800
html_src = f"<canvas id='sim_canvas' width={cwidth} height={cheight}> </canvas>"
display(HTML(html_src))
y = 20
for i in range(100):
    js_src = f"const g = document.getElementById('sim_canvas').getContext('2d');g.clearRect(0, 0, {cwidth}, {cheight});"
    circles = [draw_circle([j*1, y], 10) for j in range(600)]
    js_src += ''.join(circles)
    display(Javascript(js_src))
    sleep(.05)
    y += 1

In [ ]:
from IPython.display import display, HTML, Javascript
from time import sleep

cwidth, cheight = 800, 800
html_src = f"<canvas id='sim_canvas' width={cwidth} height={cheight}> </canvas>"
display(HTML(html_src))

Javascript("""
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

async function demo() {
    var y = 20;
    for (var i=0; i < 100; i++){
        const g = document.getElementById('sim_canvas').getContext('2d');
        g.clearRect(0, 0, 800, 800);
        for (var j=0; j <600; j++){
            g.beginPath();
            g.arc(j*1, y, 10, 0, 2 * Math.PI, false);
            g.fillStyle = 'green';
            g.fill();
            //g.lineWidth = 2;
            //g.strokeStyle = '#003300';
            g.stroke();
        }
        y = y +1;
        await sleep(50);
    }
}
demo();

""")

In [61]:
from IPython.display import display, HTML, Javascript
from time import sleep

def draw_circle(center, radius, color='#003300', linewidth=1, fill=True):
    return f"draw_circle,{center[0]},{center[1]},{radius},{color},{linewidth},{1*fill}"

def draw_line(a, b, color='#003300', linewidth=1):
    return f"draw_line,{a[0]},{a[1]},{b[0]},{b[1]},{color},{line_width}"

from ipykernel.comm import Comm
def send(comm, data):
    comm._publish_msg('comm_open', data=data, target_name=comm.target_name, target_module=comm.target_module)
    comm._closed = False

html_src = """
<canvas id='sim_canvas' width=800 height=400> </canvas>
<script>
class JSDrawing {
    constructor(canvasId) {
        this.g = document.getElementById(canvasId).getContext('2d');
    }

    drawCircle(x, y, radius, color, linewidth, fill){
        console.log([x, y, radius, color, linewidth, fill])
        this.g.beginPath();
        this.g.arc(x, y, radius, 0, 2*Math.PI, false);
        if(fill == 1){
            this.g.fillStyle = color;
            this.g.fill();
        }else{
            this.g.lineWidth = linewidth;
            this.g.strokeStyle = color;
        }
        this.g.stroke();
    }

    drawLine(ax, ay, bx, by, color, linewidth){
        this.g.beginPath();
        this.g.moveTo(ax, ay);
        this.g.lineTo(bx, by);
        this.g.lineWidth = linewidth;
        this.g.strokeStyle = color;
        this.g.stroke();
    }

    executeCommand(command){
        const args = command.split(",");
        const action = args[0];
        switch(action){
            case "draw_circle":
                this.drawCircle(args[1], args[2], args[3], args[4], args[5], args[6])
            break;
            case "draw_line":
                this.drawLine(args[1], args[2], args[3], args[4], args[5], args[6])
            break;
        }
    }

    processCommands(commands){
        commands = commands.split("|");
        this.g.clearRect(0, 0, 800, 400);
        for(let i=0; i<commands.length; i++){
            this.executeCommand(commands[i]);
        }

    }
}

const dhelper = new JSDrawing('sim_canvas');
google.colab.kernel.comms.registerTarget('absim', (comm, msg) => {dhelper.processCommands(msg.data['commands'])});
//Jupyter.notebook.kernel.comm_manager.register_target('absim', (comm, msg) => {dhelper.processCommands(msg.data['commands'])});
</script>
"""
i=1
display(HTML(html_src))

# Use comm to send a message from the kernel
comm = Comm(target_name='absim')

y = 20
for i in range(100):
    circles = [draw_circle([j*1, y], 10) for j in range(600)]
    send(comm, {'commands': '|'.join(circles)})
    sleep(.05)
    y += 1
comm.close()